Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>


В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [13]:
res = requests.get('https://habr.com/ru/')
res.status_code

200

In [22]:
def get_habr_blog(query):
    habr_blog = pd.DataFrame()
    for q in query:
        URL = 'https://habr.com/ru/search/'
        params = {'q': q}
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        posts = soup.find_all('article', class_='tm-articles-list__item')
        
        for el in posts:
            title = el.find('a', 'tm-article-snippet__title-link').text
            link = el.find('a', 'tm-article-snippet__title-link').get('href')
            date = el.find('time').get('title')
            
            row = {'date': date, 'title': title, 'link': f'https://habr.com'+link}
            habr_blog = pd.concat([habr_blog, pd.DataFrame([row])])
            
    return habr_blog.reset_index(drop=True)
res = get_habr_blog(['vst', 'синтезаторы'])
res

,date,title,link
0,"2013-12-23, 08:27",Использование слоя плана выполнения SQL запрос...,https://habr.com/ru/company/delphi/blog/206384/
1,"2017-09-19, 17:58",DevOps с Kubernetes и VSTS. Часть 2: Облачная ...,https://habr.com/ru/company/microsoft/blog/337...
2,"2017-09-12, 17:57",DevOps с Kubernetes и VSTS. Часть 1: Локальная...,https://habr.com/ru/company/microsoft/blog/337...
3,"2018-11-28, 11:34",Запускаем контейнер с юнит тестами в Azure Dev...,https://habr.com/ru/post/430958/
4,"2018-11-06, 23:50",Применение расширяемых политик Pull Request в ...,https://habr.com/ru/post/428660/
5,"2016-09-28, 14:00",Программирование&Музыка: понимаем и пишем VSTi...,https://habr.com/ru/post/311220/
6,"2020-02-18, 13:06",Быстрое создание чиптюн-музыки на открытом ПО,https://habr.com/ru/post/488436/
7,"2012-08-13, 03:37",Написание музыки в Linux: что есть прямо сейчас,https://habr.com/ru/post/149477/
8,"2014-06-06, 13:45","Создание аудиоплагинов, часть 3",https://habr.com/ru/post/225457/
9,"2010-12-26, 09:47",Бюджет домашней студии,https://habr.com/ru/post/110721/


Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи.
Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>